In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import os
import json
import pickle
from transformers import BertTokenizer
import torch
import numpy as np
import re

import warnings
warnings.filterwarnings("ignore")

from tqdm.notebook import tqdm

In [2]:
# data_path = os.environ['DATA_PATH']
data_path = "D:/Data/neural-punctuator/ted_dataset/"
os.listdir(data_path)

['.DS_Store',
 'COPYRIGHT',
 'data.pkl',
 'encoded.pkl',
 'fix_metadata.py~',
 'raw_test_texts.txt',
 'raw_train_texts.txt',
 'raw_valid_texts.txt',
 'README',
 'stats.py',
 'stats.py~',
 'targets.pkl',
 'ted_talks-10-Sep-2012.json',
 'ted_talks-25-Apr-2012.json',
 'ted_users-10-Sep-2012.json',
 'ted_users-25-Apr-2012.json',
 'test_data.pkl',
 'text.pkl',
 'texts.pkl',
 'train_data.pkl',
 'valid_data.pkl']

In [3]:
file_path = data_path + "ted_talks-25-Apr-2012.json"

In [4]:
with open(file_path, 'r', encoding='utf-8') as f:
    data = json.load(f)

In [5]:
transcripts = [d['transcript'] for d in data]

In [6]:
len(transcripts)

1149

In [7]:
ts = []

# Keep ordering
for t in transcripts:
    t = t.replace('\n', ' ')
    
    if t not in ts and len(t) > 0:
        ts.append(t)

transcripts = ts
len(transcripts)

1068

In [8]:
raw_text = '\n'.join(transcripts)
len(raw_text.split(' '))

2464687

In [9]:
import re

numbers = re.findall('[0-9]+\.[0-9]+', raw_text)
len(numbers)

428

In [10]:
action_words = re.findall('\([a-zA-Z]+\)', raw_text)
len(action_words)

7559

In [11]:
action_words = list(set(action_words))
len(action_words), action_words

(83,
 ['(Hindi)',
  '(Buzzer)',
  '(Chanting)',
  '(Tuning)',
  '(Inaudible)',
  '(Silence)',
  '(Applause)',
  '(Ringing)',
  '(Clapping)',
  '(Shouts)',
  '(Claps)',
  '(Game)',
  '(Thuds)',
  '(Buzzing)',
  '(Blip)',
  '(Coughs)',
  '(k)',
  '(Spanish)',
  '(Whinny)',
  '(Violin)',
  '(video)',
  '(Whistling)',
  '(Noise)',
  '(Recording)',
  '(Dissonance)',
  '(Screams)',
  '(Screeching)',
  '(Trumpet)',
  '(Chatter)',
  '(Laughing)',
  '(Braying)',
  '(Barking)',
  '(Crying)',
  '(Tones)',
  '(Clanging)',
  '(Bells)',
  '(Beep)',
  '(laughter)',
  '(cough)',
  '(Laughted)',
  '(Kiss)',
  '(Video)',
  '(Growling)',
  '(Honking)',
  '(Nonsense)',
  '(Dryer)',
  '(cheering)',
  '(Gunshots)',
  '(Cries)',
  '(Singing)',
  '(Roar)',
  '(Jackhammer)',
  '(Static)',
  '(Breathing)',
  '(Laughs)',
  '(Audio)',
  '(Cheering)',
  '(Sings)',
  '(Japanese)',
  '(Honk)',
  '(Cheers)',
  '(Knocks)',
  '(Music)',
  '(Laugher)',
  '(Snoring)',
  '(Thumping)',
  '(Piano)',
  '(Screaming)',
  '(Rus

In [12]:
def clean_text(text):
    escape_words = action_words

    for ew in escape_words:
        text = text.replace(ew, '')

    text = text.replace('!', '.')
    text = text.replace(':', ',')
    text = text.replace('--', ',')
    text = text.replace('-', ',')
    text = text.replace(';', '.')
    text = text.replace(' ,', ',')
    text = text.replace('♫', '')

    text = re.sub(r'--\s?--', '', text)
    text = re.sub(r'\s+', ' ', text)

    text = text.replace(' ,', ',')
    
    text = re.sub(r',\s?,', ',', text)
    
    return text

In [13]:
ts = []
for t in tqdm(transcripts):
    ts.append(clean_text(t))
transcripts = ts

In [14]:
print(transcripts[3])

I would like to talk to you about why many e,health projects fail. And I really think that the most important thing of it is that we stopped listening to patients. And one of the things we did at Radboud University is we appointed a chief listening officer. Not in a very scientific way, she puts up a little cup of coffee or cup of tea and asks patients, family, relatives, "What's up? How could we help you?" And we think, we like to think, that this is one of the major problems why all, maybe not all, but most of the e,health projects fail, since we stopped listening. This is my WiFi scale. It's a very simple thing. It's got one knob, on/off. And every morning I hop on it. And yes, I've got a challenge, as you might see. And I put my challenge on 95 kg. But the thing is that it's made this simple that whenever I hop on, it sends my data through Google Health as well. And it's collected by my general practitioner as well, so he can see what's my problem in weight, not on the very moment 

In [15]:
albert_pretrained = torch.hub.load('huggingface/pytorch-transformers', 'model', 'albert-base-v1')
albert_tokenizer = torch.hub.load('huggingface/pytorch-transformers', 'tokenizer', 'albert-base-v1')
albert_pretrained.eval()

Using cache found in C:\Users\gbenc/.cache\torch\hub\huggingface_pytorch-transformers_master
Using cache found in C:\Users\gbenc/.cache\torch\hub\huggingface_pytorch-transformers_master


AlbertModel(
  (embeddings): AlbertEmbeddings(
    (word_embeddings): Embedding(30000, 128, padding_idx=0)
    (position_embeddings): Embedding(512, 128)
    (token_type_embeddings): Embedding(2, 128)
    (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): AlbertTransformer(
    (embedding_hidden_mapping_in): Linear(in_features=128, out_features=768, bias=True)
    (albert_layer_groups): ModuleList(
      (0): AlbertLayerGroup(
        (albert_layers): ModuleList(
          (0): AlbertLayer(
            (full_layer_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (attention): AlbertAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (attention_dropout): Dropout(p=0.1, inplace=False)
  

In [16]:
# %%timeit
# albert_tokenizer.tokenize(" , ! ? . ... .. asd. qwe.. tyu, uio , asdfs")

In [17]:
# %%timeit
# albert_tokenizer.encode(" , ! ? . ... .. asd. qwe.. tyu, uio , asdfs")

In [18]:
import re
albert_tokenizer.tokenize(re.sub("[0-9]+.[0-9]+", "<unk>", "A pi value is 3.14 and which is near 3."))

['▁a',
 '▁pi',
 '▁value',
 '▁is',
 '<unk>',
 '▁and',
 '▁which',
 '▁is',
 '▁near',
 '▁3',
 '.']

In [19]:
target_ids = (9, 60, 15)
albert_tokenizer.encode(".?,")

[2, 13, 9, 60, 15, 3]

In [20]:
[albert_tokenizer._convert_id_to_token(i) for i in target_ids]

['.', '?', ',']

In [21]:
encoded_texts = []
for t in tqdm(transcripts):
    encoded_texts.append(albert_tokenizer.encode(t))

with open(data_path + 'encoded.pkl', 'wb') as f:
    pickle.dump(encoded_texts, f)

Token indices sequence length is longer than the specified maximum sequence length for this model (3153 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1022 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (3561 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1063 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (4292 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length fo

Token indices sequence length is longer than the specified maximum sequence length for this model (5971 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (3641 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (2806 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (2906 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (3308 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length fo

Token indices sequence length is longer than the specified maximum sequence length for this model (531 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1520 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (4097 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (4486 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (4401 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for

Token indices sequence length is longer than the specified maximum sequence length for this model (1026 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (3738 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (2992 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1052 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (3967 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length fo

Token indices sequence length is longer than the specified maximum sequence length for this model (2810 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (2174 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (2938 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (5116 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (3664 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length fo

...

Token indices sequence length is longer than the specified maximum sequence length for this model (847 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (3335 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (2177 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (4618 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1865 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for

Token indices sequence length is longer than the specified maximum sequence length for this model (1881 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (4602 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (5098 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (586 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (3610 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for

Token indices sequence length is longer than the specified maximum sequence length for this model (2007 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (2708 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (2862 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (3249 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (854 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for

Token indices sequence length is longer than the specified maximum sequence length for this model (1838 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (603 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (549 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1457 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (871 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for t

Token indices sequence length is longer than the specified maximum sequence length for this model (2122 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1448 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (3039 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (2927 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (2771 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length fo

In [22]:
# with open(data_path + 'encoded.pkl', 'rb') as f:
#     encoded_texts = pickle.load(f)

In [23]:
id2target = {-1: 0,
              9: 1, # .
              60: 2, # ?
              15: 3,  # ,
              -2: -1, # will be masked
             }
target2id = {value: key for key, value in id2target.items()}
    
    
def create_target(encoded):
    targets = []
    text = []

    target = -2 # Always mask after [CLS] token

    text.append(encoded[0])
    idx = 1
    while idx < len(encoded):
        word = encoded[idx]
        if word in id2target.keys():
            
            if word in (9,15) and \
            albert_tokenizer._convert_id_to_token(encoded[idx-1]).lstrip('▁').isnumeric() and \
            albert_tokenizer._convert_id_to_token(encoded[idx+1]).isnumeric():
                targets.append(-2) # prev number
#                 print(-2)
#                 print(f"{albert_tokenizer._convert_id_to_token(word):15}\t", end="")
                text.append(word)                
                target = -2 # '. or ,'
            else:
                target = word
        else:
            if not albert_tokenizer._convert_id_to_token(word).startswith('▁'):
                target = -2
            targets.append(target)
#             print(target)
#             print(f"{albert_tokenizer._convert_id_to_token(word):15}\t", end="")
            target = -1
            text.append(word)
        idx += 1

    targets.append(target)

    targets = [id2target[t] for t in targets]

    return text, targets

In [24]:
texts = []
targets = []
for text in tqdm(encoded_texts):
    text, target = create_target(text)
    texts.append(text)
    targets.append(target)

In [25]:
search = '[0-9]+\.[0-9]+'
encoded_joint_text = ' '.join(albert_tokenizer.decode(et) for et in texts)
idxs = [m.start() for m in re.finditer('(?=%s)(?!.{1,%d}%s)' % (search, len(search)-1, search), encoded_joint_text)]
len(idxs)

401

In [26]:
for et in texts:
    for i in range(len(et)):
        if et[i] == 9: # .
            if albert_tokenizer.decode(et[i-1]).isnumeric() and albert_tokenizer.decode(et[i+1]).isnumeric() and not albert_tokenizer._convert_id_to_token(et[i+1]).startswith('▁'):
                print([albert_tokenizer._convert_id_to_token(eti) for eti in et[i-1:i+2]], albert_tokenizer.decode(et[i-1:i+2]))

['▁0', '.', '1'] 0.1
['▁7', '.', '95'] 7.95
['▁2', '.', '0'] 2.0
['▁1', '.', '2'] 1.2
['▁1', '.', '2'] 1.2
['▁39', '.', '5'] 39.5
['▁2', '.', '0'] 2.0
['▁1', '.', '8'] 1.8
['▁99', '.', '7'] 99.7
['▁1', '.', '1'] 1.1
['▁1', '.', '1'] 1.1
['▁1', '.', '5'] 1.5
['▁1', '.', '7'] 1.7
['▁1', '.', '4'] 1.4
['▁0', '.', '5'] 0.5
['▁8', '.', '1'] 8.1
['▁8', '.', '2'] 8.2
['▁8', '.', '3'] 8.3
['▁1', '.', '5'] 1.5
['▁2', '.', '3'] 2.3
['▁3', '.', '5'] 3.5
['▁27', '.', '5'] 27.5
['▁2', '.', '5'] 2.5
['▁1', '.', '5'] 1.5
['▁1', '.', '7'] 1.7
['▁1', '.', '2'] 1.2
['▁1', '.', '2'] 1.2
['▁13', '.', '7'] 13.7
['▁13', '.', '7'] 13.7
['▁0', '.', '03'] 0.03
['▁1', '.', '1'] 1.1
['▁1', '.', '5'] 1.5
['▁1', '.', '5'] 1.5
['▁1', '.', '4'] 1.4
['▁1', '.', '5'] 1.5
['▁1', '.', '5'] 1.5
['▁1', '.', '2'] 1.2
['▁4', '.', '5'] 4.5
['▁2', '.', '5'] 2.5
['▁2', '.', '5'] 2.5
['▁1', '.', '2'] 1.2
['▁1', '.', '5'] 1.5
['▁1', '.', '3'] 1.3
['▁1', '.', '7'] 1.7
['▁3', '.', '2'] 3.2
['▁1', '.', '7'] 1.7
['▁25', '.', '3'] 25

In [27]:
for te, ta in zip(texts[2][:10000], targets[2]):
    print(f"{albert_tokenizer._convert_id_to_token(te):15}\t{ta}")

[CLS]          	-1
▁penelope      	0
▁jag           	-1
es             	-1
sar            	0
▁cha           	-1
ffer           	3
▁i             	0
▁was           	0
▁going         	0
▁to            	0
▁ask           	0
▁if            	0
▁there         	-1
'              	-1
s              	0
▁a             	0
▁doctor        	0
▁in            	0
▁the           	0
▁house         	1
▁no            	3
▁i             	-1
'              	-1
m              	0
▁just          	0
▁joking        	1
▁it            	-1
'              	-1
s              	0
▁interesting   	3
▁because       	0
▁it            	0
▁was           	0
▁six           	0
▁years         	0
▁ago           	0
▁when          	0
▁i             	0
▁was           	0
▁pregnant      	0
▁with          	0
▁my            	0
▁first         	0
▁child         	0
▁that          	0
▁i             	0
▁discovered    	0
▁that          	0
▁the           	0
▁most          	0
▁commonly      	0
▁used          	0
▁pre           	-1
serv           	-

▁environment   	1
▁p             	-1
jc             	3
▁so            	0
▁speaking      	0
▁of            	0
▁estrogen      	3
▁one           	0
▁of            	0
▁the           	0
▁other         	0
▁compounds     	0
▁that          	0
▁tyrone        	0
▁talks         	0
▁about         	0
▁in            	0
▁the           	0
▁film          	0
▁is            	0
▁something     	0
▁called        	0
▁bis           	-1
phenol         	0
▁a             	3
▁b             	-1
pa             	3
▁which         	0
▁has           	0
▁been          	0
▁in            	0
▁the           	0
▁news          	0
▁recently      	1
▁it            	-1
'              	-1
s              	0
▁a             	0
▁plastic       	-1
izer           	1
▁it            	-1
'              	-1
s              	0
▁a             	0
▁compound      	0
▁that          	-1
'              	-1
s              	0
▁found         	0
▁in            	0
▁poly          	-1
carbon         	-1
ate            	0
▁plastic       	3
▁which         	

lim            	-1
ey             	1
▁never         	0
▁heard         	0
▁of            	0
▁it            	1
▁p             	-1
jc             	3
▁and           	0
▁neither       	0
▁had           	0
▁i             	0
▁actually      	0
▁before        	0
▁i             	0
▁started       	0
▁making        	0
▁this          	0
▁film          	1
▁and           	0
▁so            	0
▁when          	0
▁you           	0
▁realize       	0
▁that          	0
▁chemicals     	0
▁can           	0
▁pass          	0
▁the           	0
▁place         	-1
nta            	0
▁and           	0
▁go            	0
▁into          	0
▁your          	0
▁un            	-1
born           	0
▁child         	3
▁it            	0
▁made          	0
▁me            	0
▁start         	0
▁to            	0
▁think         	3
▁what          	0
▁would         	0
▁my            	0
▁fetus         	0
▁say           	0
▁to            	0
▁me            	2
▁what          	0
▁would         	0
▁our           	0
▁un            	-1
born 

▁those         	0
▁re            	-1
per            	-1
cus            	-1
s              	-1
ions           	3
▁but           	0
▁we            	0
▁are           	0
▁so            	0
▁ignorant      	0
▁of            	0
▁this          	0
▁sense         	0
▁of            	0
▁what          	0
▁happens       	0
▁when          	0
▁we            	0
▁put           	0
▁things        	3
▁or            	0
▁things        	0
▁are           	0
▁put           	0
▁into          	0
▁our           	0
▁bodies        	1
▁and           	0
▁it            	-1
'              	-1
s              	0
▁my            	0
▁feeling       	0
▁and           	0
▁it            	-1
'              	-1
s              	0
▁my            	0
▁urging        	0
▁being         	0
▁here          	0
▁to            	0
▁know          	0
▁that          	3
▁as            	0
▁we            	0
▁women         	0
▁move          	0
▁forward       	0
▁as            	0
▁the           	0
▁              	-1
commun         	-1
ic             	-1

In [28]:
n = -120
test_n = -6#int(.96*len(texts))
train_texts = texts[:n]
train_targets = targets[:n]
valid_texts = texts[n:test_n]
valid_targets = targets[n:test_n]
test_texts = texts[test_n:]
test_targets = targets[test_n:]
len(train_texts), len(valid_texts), len(test_texts)

(948, 114, 6)

In [29]:
len(' '.join(raw_text.split('\n')[:n]).split(' ')), \
len(' '.join(raw_text.split('\n')[n:test_n]).split(' ')), \
len(' '.join(raw_text.split('\n')[test_n:]).split(' '))

(2222752, 230236, 12766)

In [30]:
with open(data_path + 'raw_train_texts.txt', 'w', encoding='utf-8') as f:
    f.write('\n'.join(raw_text.split('\n')[:n]) + "\n")
with open(data_path + 'raw_valid_texts.txt', 'w', encoding='utf-8') as f:
    f.write('\n'.join(raw_text.split('\n')[n:test_n]) + "\n")
with open(data_path + 'raw_test_texts.txt', 'w', encoding='utf-8') as f:
    f.write('\n'.join(raw_text.split('\n')[test_n:]) + "\n")

In [31]:
from collections import Counter
c = Counter((t for targets in valid_targets for t in targets))
[c[i] for i in (1,2,3,0,-1)]

[13721, 882, 15344, 197884, 29627]

In [32]:
sum(len(t) for t in valid_texts)

257458

In [33]:
np.sum(np.array(valid_targets[0]) != -1)

3258

In [34]:
sum((np.array(vt) != -1).sum() for vt in valid_targets)

227831

In [35]:
with open(data_path + 'train_data.pkl', 'wb') as f:
    pickle.dump((train_texts, train_targets), f)
with open(data_path + 'valid_data.pkl', 'wb') as f:
    pickle.dump((valid_texts, valid_targets), f)
with open(data_path + 'test_data.pkl', 'wb') as f:
    pickle.dump((test_texts, test_targets), f)

# with open(data_path + 'train_data.pkl', 'rb') as f:
#     train_texts, train_targets = pickle.load(f)

In [36]:
from collections import Counter
c = Counter((t for targets in valid_targets for t in targets))
c

Counter({-1: 29627, 0: 197884, 1: 13721, 3: 15344, 2: 882})

In [37]:
def get_weights_for_sampler(train_data):
    # used because of the imbalanced dataset
    train_data['sector_int'], _ = pd.factorize(train_data['sector'])

    class_sample_count = np.array(
        [len(np.where(train_data['sector_int'] == t)[0]) for t in np.unique(train_data['sector_int'])])
    weight = 1. / class_sample_count
    samples_weight = np.array([weight[t] for t in train_data['sector_int']])

    samples_weight = torch.from_numpy(samples_weight)
    samples_weig

In [38]:
targets = np.array([t for targets in train_targets for t in targets])
targets.shape, targets[:2]

((2435170,), array([-1,  0]))

In [39]:
output_dim = 4

weights = np.zeros((output_dim, ))
for t in range(output_dim):
    count = (targets == t).sum()
    weights[t] = count

weights /= weights.sum()
weights

array([0.85566442, 0.05960905, 0.00448369, 0.08024283])

In [40]:
c

Counter({-1: 29627, 0: 197884, 1: 13721, 3: 15344, 2: 882})

In [41]:
# .?,
s = 0
weights = []
for key in range(output_dim):
    value = c[key]
    s += value
for key in range(output_dim):
    value = c[key]
    print(key, target2id[key], albert_tokenizer.decode(target2id[key]) if key > 0 else "\"\"", value, value/s)
    weights.append(s/value)

0 -1 "" 197884 0.868556078847918
1 9 . 13721 0.06022446462509492
2 60 ? 882 0.0038712905618638375
3 15 , 15344 0.06734816596512327


In [42]:
import numpy as np
weights = np.array(weights)
weights /= weights.sum()
weights

array([0.00395763, 0.0570768 , 0.88792603, 0.05103954])